In [1]:
include("../src/Julia.jl")

objc[851]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[851]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[851]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[851]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/ipm_grid3_8000_all/";

In [4]:
graphs = readdir(gpath);
len = length(graphs)
graphs = graphs[ceil(Int64,len * 0.2)+1:ceil(Int64,len * 0.4)];

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
#         x = readFromFile(path * "x.vec");
        x = zeros(a.n)
        b = readFromFile(path * "b.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        bdef = copy(b)
        bdef = bdef - mean(bdef);
        
        numIts = 100;
        @time for i in 1:5
            @time F = treeSolver(tree[i])
            @time F(bdef);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, bdef, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - bdef) / norm(bdef)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now using higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                
                xHP = zeros(BigFloat, a.n)
                
                bHP = toHighPrecision(b; precision=hp);
                bHP = bHP - mean(bHP)
                
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

  0.485902 seconds (639.99 k allocations: 29.183 MB, 1.52% gc time)
  0.154353 seconds (245.33 k allocations: 6.714 MB, 4.14% gc time)
  0.877458 seconds (15.63 M allocations: 386.092 MB, 4.27% gc time)
 51.148835 seconds (262.09 M allocations: 6.629 GB, 37.09% gc time)
 58.095996 seconds (261.91 M allocations: 7.264 GB, 38.18% gc time)
 63.947750 seconds (261.91 M allocations: 8.549 GB, 37.12% gc time)
 76.728106 seconds (261.91 M allocations: 11.118 GB, 36.48% gc time)
 86.022148 seconds (261.91 M allocations: 16.256 GB, 34.91% gc time)
  0.019194 seconds (40.19 k allocations: 4.597 MB)
  0.005310 seconds (152.67 k allocations: 2.819 MB)
  0.812794 seconds (15.43 M allocations: 376.600 MB, 23.49% gc time)
 52.198141 seconds (261.93 M allocations: 6.622 GB, 37.44% gc time)
 57.126652 seconds (261.93 M allocations: 7.264 GB, 38.00% gc time)
 64.450398 seconds (261.93 M allocations: 8.549 GB, 37.41% gc time)
 75.584486 seconds (261.93 M allocations: 11.118 GB, 36.55% gc time)
 88.818315